<a href="https://colab.research.google.com/github/lbuerger/IANNwTF/blob/main/homework3/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [137]:
ds = tfds.load('genomics_ood', as_supervised=True)
train_ds = tfds.load('genomics_ood', as_supervised =True, split='train[:1000]')
test_ds = tfds.load('genomics_ood', as_supervised =True, split='test[:100]')
ds['test']
print_ds(ds['test'])

(<tf.Tensor: shape=(), dtype=string, numpy=b'CTCCGCATGATCGAGGAGCCGCACCTTGCAGTGCGGGCACAGCAGCTTCACGAGGCGCTGGCTGATGAGCCCGGAGATCACCGTGTGATCGGTGATCATCCGCGCGTGCAGGCCGAGATCGATCAGGCGATCGGCGATCGCGAGCGCGCTGTTCGCGTGCACGGTCGTCCACACCTGATGGCCCGTCATCGACGCGCGCAGCGCCGTCTGCCCGGACGCGCGATCGCGGATCTCGCCGATCATGATCGTG'>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)


In [39]:
def print_ds(ds):
  for i in ds:
    print(i)
    break;


In [130]:
train_ds = train_ds.prefetch(128)

In [134]:
train_ds_slice = tf.data.Dataset.from_tensor_slices([train_ds])
#train_ds_slice = train_ds_slice.map(lambda dat : tf.reshape(dat[0],(-1,)))
train_ds_slice.map(lambda x : simplify_ds(x))
train_ds_slice

<_VariantDataset shapes: ((), ()), types: (tf.string, tf.int64)>
simply (<tf.Tensor 'args_1:0' shape=() dtype=string>, <tf.Tensor 'args_2:0' shape=() dtype=int64>)


<TensorSliceDataset shapes: DatasetSpec((TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None)), TensorShape([])), types: DatasetSpec((TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None)), TensorShape([]))>

In [99]:
def onehotify(tensor):
  vocab = {'A':'1', 'C': '2', 'G':'3', 'T':'0'}
  for key in vocab.keys():
    tensor = tf.strings.regex_replace(tensor, key, vocab[key])
  split = tf.strings.bytes_split(tensor)
  labels = tf.cast(tf.strings.to_number(split), tf.uint8)
  onehot = tf.one_hot(labels, 4)
  onehot = tf.reshape(onehot, (-1,))  
  return onehot

In [91]:
def simplify_ds(data):
  print(data)
  for i in data:
    print('simply', i)
  return data

In [138]:
def make_usefull(ds):
  data = []
  label = []
  for i in ds:
    data.append(onehotify(i[0]))
    label.append(tf.one_hot(i[1],10))
  data_ds = tf.data.Dataset.from_tensor_slices(data)
  label_ds = tf.data.Dataset.from_tensor_slices(label)
  return tf.data.Dataset.zip((data_ds, label_ds))

In [ ]:
make_usefull(ds['test'])